<a href="https://colab.research.google.com/github/TeAmP0is0N/Hostility-Detection/blob/master/models/BERT_Embeddings_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import pandas as pd
import numpy as np
import pickle
import copy
import re
import matplotlib.pyplot as plt
from sklearn.externals import joblib

from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [ ]:
pd.options.display.max_colwidth = 0

In [ ]:
train = pd.read_csv("/gdrive/My Drive/IIITD Fake News Project/Data Set/Constraint 21/Hostility Approach/Complete Dataset/Final Processed Dataset/70-10-20/train.csv")
valid = pd.read_csv("/gdrive/My Drive/IIITD Fake News Project/Data Set/Constraint 21/Hostility Approach/Complete Dataset/Final Processed Dataset/70-10-20/valid.csv")
test  = pd.read_csv("/gdrive/My Drive/IIITD Fake News Project/Data Set/Constraint 21/Hostility Approach/Complete Dataset/Final Processed Dataset/70-10-20/test.csv")

In [ ]:
print(train.shape)
print(valid.shape)
print(test.shape)

(5728, 3)
(811, 3)
(1653, 3)


In [ ]:
train = train.dropna()
valid = valid.dropna()
test  = test.dropna()

print(train.shape)
print(valid.shape)
print(test.shape)

(5728, 3)
(811, 3)
(1653, 3)


In [ ]:
train.head(5)

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है \n\n🙏 जय श्री राम 🙏,"hate,offensive"
1,2,"सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नई-नई स्कीमें लाती रहती है, ताकि उन पर ज्यादा आर्थिक बोझ न पड़े.\n\nhttps://t.co/8iy2MJSBAs",non-hostile
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील दीपेश को सुशांत की हत्या के दिन ही क्यों याद आई? देखिए 'पूछता है भारत' अर्नब के साथ रिपब्लिक भारत पर #LIVE : https://t.co/G945HvzM0Z https://t.co/KfH7xF1IdM",non-hostile
3,4,@prabhav218 साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है। सच्चाई यह है कि यह बराबर अधिकार नहीं देता है।,"defamation,offensive"
4,5,#unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स जारी\n\n- 7 सितंबर से देशभर में मेट्रो सेवा शुरु होगी\n- 21 सितंबर के बाद रैलियों और बाकी फंक्शन में 100 लोगों को इजाजत\n- कंटेनमेंट जोन में कोई छूट नहीं\n- सिनेमाहॉल अभी बंद रहेंगे\n- 9 से 12वीं के छात्र 21 सितंबर के बाद स्कूल जा सकेंगे. https://t.co/4e6lysg0VR,non-hostile


In [ ]:
def preprocess(text):
    text = text.lower()
    url = r'http\S+'
    text = re.sub(url, ' ', text, flags=re.MULTILINE)
    emoji = re.compile("["         u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002500-\U00002BEF"  # chinese char
                                   u"\U00002702-\U000027B0"
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   u"\U0001f926-\U0001f937"
                                   u"\U00010000-\U0010ffff"
                                   u"\u2640-\u2642"
                                   u"\u2600-\u2B55"
                                   u"\u200d"
                                   u"\u23cf"
                                   u"\u23e9"
                                   u"\u231a"
                                   u"\ufe0f"  # dingbats
                                   u"\u3030"
                                   "]+", flags=re.UNICODE)
    text =  emoji.sub(r'', text)
    text = ' '.join([word[1:] if word[0] == '#' else word for word in text.split()])

    text = text.split()
    exclude_char_list = ['@','rt','|','%', '[', '(', '{', '}', ']', ')','url','URL']
    text = [word for word in text if all(ch not in word for ch in exclude_char_list)]
    text = ' '.join(text)
    return text

In [ ]:
def assign_labels(text):

    text   = text.split(',')
    labels = [0,0,0,0,0]
    

    for item in text:
        if item=='defamation':
            labels[0] = 1
            break
        
    for item in text:
        if item=='fake':
            labels[1] = 1
            break

    for item in text:
        if item=='hate':
            labels[2] = 1
            break

    for item in text:
        if item=='offensive':
            labels[3] = 1
            break
        
    for item in text:
        if item=='non-hostile':
            labels[4] = 1
            break    
    
    return labels

In [ ]:
def prepare_data(df):
    
    for i in range(len(df)):    
        df.at[i,'Post'] = preprocess(df['Post'][i])
        df.at[i, 'Labels Set'] = assign_labels(df['Labels Set'][i])

    return df


In [ ]:
train = prepare_data(train)
valid = prepare_data(valid)
test  = prepare_data(test)

print(train.shape)
print(valid.shape)
print(test.shape)

(5728, 3)
(811, 3)
(1653, 3)


In [ ]:
train = train.append(valid)
train

,Unique ID,Post,Labels Set
0,1,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है जय श्री राम,"[0, 0, 1, 1, 0]"
1,2,"सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नई-नई स्कीमें लाती रहती है, ताकि उन पर ज्यादा आर्थिक बोझ न पड़े.","[0, 0, 0, 0, 1]"
2,3,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील दीपेश को सुशांत की हत्या के दिन ही क्यों याद आई? देखिए 'पूछता है भारत' अर्नब के साथ रिपब्लिक भारत पर live :","[0, 0, 0, 0, 1]"
3,4,साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है। सच्चाई यह है कि यह बराबर अधिकार नहीं देता है।,"[1, 0, 0, 1, 0]"
4,5,unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स जारी - 7 सितंबर से देशभर में मेट्रो सेवा शुरु होगी - 21 सितंबर के बाद रैलियों और बाकी फंक्शन में 100 लोगों को इजाजत - कंटेनमेंट जोन में कोई छूट नहीं - सिनेमाहॉल अभी बंद रहेंगे - 9 से 12वीं के छात्र 21 सितंबर के बाद स्कूल जा सकेंगे.,"[0, 0, 0, 0, 1]"
...,...,...,...
806,807,चंद फेंके हुए टुकड़ों के लिए हमेशा ही देश से गद्दारी को भौंकने तैयार ड्रग्स के नशे में पूरी तरह खोखले हो चुके दाऊद के गुलाम देशद्रोही गद्दार फिल्मी भांडों के भोंकने को भी मीडिया ब्रेकिंग न्यूज़ बना इन दो कौड़ी के बिकाऊ नशेड़ी गद्दार कुत्तों को हीरो बना देती है देश की विडंबना,"[0, 0, 1, 0, 0]"
807,808,आँखों ही आँखों में इशारा हो गया बैठे बैठे जीने का सहारा हो गया नज़रों की इस मारक क्षमता को क्या कहगे।,"[1, 0, 0, 0, 0]"
808,809,"औरंगाबाद में फिर धीरे-धीरे पांव पसार रहा कोरोना संक्रमण, पिछले 6 दिनों में मिले इतने नए मरीज bihar corona aurangabad","[0, 0, 0, 0, 1]"
809,810,"बाइस्कोप: कादर के संवादों का कमाल, डिंपल की कलम का धमाल और तीन बत्ती के जग्गू का नाना संग असली बवाल bioscope angaar shashilalnair dimplekapadia","[0, 0, 0, 0, 1]"


In [ ]:
train = train.drop(columns='Unique ID', axis =1)
train = train.reset_index(drop=True)
train

,Post,Labels Set
0,मेरे देश के हिन्दु बहुत निराले है। कुछ तो पक्के राम भक्त है और कुछ बाबर के साले है जय श्री राम,"[0, 0, 1, 1, 0]"
1,"सरकार हमेशा से किसानों की कमाई को बढ़ाने के लिए नई-नई स्कीमें लाती रहती है, ताकि उन पर ज्यादा आर्थिक बोझ न पड़े.","[0, 0, 0, 0, 1]"
2,"सुशांत ने जो बिजनेस डील 9 जून को की थी, वो डील दीपेश को सुशांत की हत्या के दिन ही क्यों याद आई? देखिए 'पूछता है भारत' अर्नब के साथ रिपब्लिक भारत पर live :","[0, 0, 0, 0, 1]"
3,साले जेएनयू छाप कमिने लोग हिन्दुओं को यह कहते है की संविधान सबको बराबर अधिकार देता है। सच्चाई यह है कि यह बराबर अधिकार नहीं देता है।,"[1, 0, 0, 1, 0]"
4,unlock4guidelines - अनलॉक-4 के लिए गाइडलाइन्स जारी - 7 सितंबर से देशभर में मेट्रो सेवा शुरु होगी - 21 सितंबर के बाद रैलियों और बाकी फंक्शन में 100 लोगों को इजाजत - कंटेनमेंट जोन में कोई छूट नहीं - सिनेमाहॉल अभी बंद रहेंगे - 9 से 12वीं के छात्र 21 सितंबर के बाद स्कूल जा सकेंगे.,"[0, 0, 0, 0, 1]"
...,...,...
6534,चंद फेंके हुए टुकड़ों के लिए हमेशा ही देश से गद्दारी को भौंकने तैयार ड्रग्स के नशे में पूरी तरह खोखले हो चुके दाऊद के गुलाम देशद्रोही गद्दार फिल्मी भांडों के भोंकने को भी मीडिया ब्रेकिंग न्यूज़ बना इन दो कौड़ी के बिकाऊ नशेड़ी गद्दार कुत्तों को हीरो बना देती है देश की विडंबना,"[0, 0, 1, 0, 0]"
6535,आँखों ही आँखों में इशारा हो गया बैठे बैठे जीने का सहारा हो गया नज़रों की इस मारक क्षमता को क्या कहगे।,"[1, 0, 0, 0, 0]"
6536,"औरंगाबाद में फिर धीरे-धीरे पांव पसार रहा कोरोना संक्रमण, पिछले 6 दिनों में मिले इतने नए मरीज bihar corona aurangabad","[0, 0, 0, 0, 1]"
6537,"बाइस्कोप: कादर के संवादों का कमाल, डिंपल की कलम का धमाल और तीन बत्ती के जग्गू का नाना संग असली बवाल bioscope angaar shashilalnair dimplekapadia","[0, 0, 0, 0, 1]"


In [ ]:
train.iloc[5816]

Post          विशेषज्ञ सलाह  
Labels Set    [0, 0, 0, 0, 1]
Name: 5816, dtype: object

## Getting BERT Embeddings

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer

bert_cls  = SentenceTransformer('bert-base-nli-cls-token')
bert_mean = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
#Getting BERT CLS Embeddings
train_bert_cls = pd.DataFrame(bert_cls.encode(train['Post']))
test_bert_cls  = pd.DataFrame(bert_cls.encode(test['Post']))

In [ ]:
train_bert_cls = pd.concat([train_bert_cls, train['Labels Set']], axis=1)
test_bert_cls  = pd.concat([test_bert_cls,  test['Labels Set']],  axis=1)

In [ ]:
print(train_bert_cls.shape)
print(test_bert_cls.shape)

(6539, 769)
(1653, 769)


In [ ]:
# #Getting BERT Mean Embeddings
# train_bert_mean = bert_mean.encode(x_train)
# test_bert_mean  = bert_mean.encode(x_test)

In [ ]:
# #Dumping Embedding Features DataFrame

# path_temp = r"/gdrive/My Drive/3rd Sem/MTech CSE 3/Thesis/IIITD Fake News Project/Data Set/Embeddings Datasets/BERT_CLS/train_df.pkl"
# f = open(path_temp,'wb')
# pickle.dump(train_bert_cls,f)
# f.close()


# path_temp = r"/gdrive/My Drive/3rd Sem/MTech CSE 3/Thesis/IIITD Fake News Project/Data Set/Embeddings Datasets/BERT_CLS/test_df.pkl"
# f = open(path_temp,'wb')
# pickle.dump(test_bert_cls,f)
# f.close()

In [ ]:
#Loading Embedding Features DataFrame

path_temp = r"/gdrive/My Drive/IIITD Fake News Project/Data Set/Embeddings Datasets/BERT_CLS/train_df.pkl"
f = open(path_temp,'rb')
train_bert_cls = pickle.load(f)
f.close()

path_temp = r"/gdrive/My Drive/IIITD Fake News Project/Data Set/Embeddings Datasets/BERT_CLS/test_df.pkl"
f = open(path_temp,'rb')
test_bert_cls = pickle.load(f)
f.close()


In [ ]:
train_bert_cls

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,Labels Set
0,-0.144744,0.753920,-0.370921,-0.611838,0.458139,0.191976,1.445014,-0.574958,-0.530926,0.478863,0.360358,1.185349,-0.117227,0.353543,0.289014,0.240586,0.918327,-0.305945,0.875681,-0.444604,-0.467921,0.422519,0.080013,0.269675,0.482003,0.725526,0.351850,-0.525961,-0.860559,0.508275,0.489205,0.122160,-0.546525,0.099349,0.583576,-0.010702,0.248857,-0.396506,-0.056162,-0.435263,...,-0.491767,-0.804737,0.230529,-0.513869,-0.527120,1.364753,0.334724,-1.164227,-0.155990,-0.386536,-0.769295,-0.651128,-0.073439,-0.259624,-1.097570,-0.121238,-0.471517,0.338178,-0.808320,-0.136119,1.095405,0.175688,0.775150,0.348087,0.170919,-1.307852,-0.038388,-0.590353,0.536165,-0.006736,-0.119820,-0.907450,-0.134403,0.280859,-1.141616,0.078590,0.189792,0.589307,-0.206571,"[0, 0, 1, 1, 0]"
1,-0.519353,0.546359,-0.319189,-0.343406,0.505719,0.373533,1.298511,-0.344353,-0.530590,0.271636,0.292417,1.001168,-0.124076,0.185682,0.657651,0.287772,0.958941,-0.204964,0.836533,-0.753627,-0.514037,0.309644,0.023796,0.073000,0.524523,0.696313,0.141873,-0.547655,-0.631862,0.541703,0.285761,0.618741,-1.041801,0.418552,0.797897,0.144787,0.104293,-0.449161,-0.049456,-0.075865,...,-0.595334,-0.358078,0.063206,-0.460307,-0.577727,1.469891,0.279616,-0.914689,-0.035396,-0.235962,-0.706077,-0.889545,0.015650,-0.178160,-0.755991,-0.091082,-0.379350,0.514821,-0.688070,-0.505213,0.880837,0.252368,0.674633,0.443868,-0.087992,-1.137267,-0.249685,-0.872141,0.609409,-0.215903,-0.286818,-0.965113,-0.154222,-0.008278,-0.969820,0.197929,0.145973,0.389509,-0.169360,"[0, 0, 0, 0, 1]"
2,-0.035937,0.593653,-0.028129,-0.481018,0.558263,0.194883,1.548601,-0.022254,-0.594360,0.296011,0.015633,1.396214,-0.270457,-0.021687,0.463896,0.366739,0.755320,-0.512736,0.867576,-0.751097,-0.319374,0.014895,0.010531,0.331701,0.465910,0.807952,0.127065,-0.334813,-0.748627,0.587821,0.072867,0.182084,-0.688813,0.271975,0.671757,0.292573,-0.198882,-0.176110,-0.048825,-0.067558,...,-0.510239,-0.502761,0.298681,-0.448650,-0.755805,1.287324,0.211610,-0.989575,-0.142912,-0.291308,-0.932961,-0.552130,-0.138239,-0.629656,-1.007847,0.220976,-0.093494,0.453248,-1.047838,-0.551566,1.155589,0.186428,0.486323,0.205587,-0.226365,-1.205978,0.144138,-0.787474,0.391423,-0.125152,0.108841,-0.923864,-0.107842,-0.134367,-0.918103,0.002269,0.073970,0.471125,-0.300663,"[0, 0, 0, 0, 1]"
3,-0.330956,0.835987,-0.244033,-0.472148,0.451524,0.050984,1.388158,-0.680034,-0.341974,0.238754,0.445741,1.286209,-0.095728,0.354333,0.181936,0.264372,0.966280,-0.244654,0.609970,-0.467251,-0.329228,0.438032,0.184911,0.297807,0.631634,0.734913,0.083992,-0.646495,-0.963654,0.812570,0.447330,0.327190,-0.721536,0.113138,0.413360,0.232470,0.170344,-0.340805,0.104577,-0.505388,...,-0.522492,-0.710157,0.262582,-0.504804,-0.529693,1.097748,0.288841,-1.240061,-0.046536,-0.321174,-0.795053,-0.756293,-0.318385,-0.488590,-1.142724,-0.139520,-0.622172,0.271261,-0.733643,-0.126173,1.047742,0.049972,0.877653,0.196111,0.136220,-1.186355,-0.022802,-0.559142,0.678789,0.160011,-0.122833,-1.104039,-0.117944,0.202165,-1.308372,0.128998,0.035356,0.759933,-0.158489,"[1, 0, 0, 1, 0]"
4,-0.572113,0.866568,-0.252675,-0.305324,0.361157,0.288053,1.026320,-0.391716,-0.831329,0.483821,0.015659,1.239518,-0.120939,0.212706,0.157652,0.540641,0.962199,-0.336899,0.931267,-0.577887,0.009949,0.440167,0.191770,0.207748,0.503330,0.932699,0.099071,-0.323963,-0.578352,0.675329,0.353921,0.533808,-0.687090,0.458181,0.901226,-0.141238,-0.074874,-0.561047,0.063945,-0.075129,...,-0.631871,0.020675,-0.095194,-0.008814,-0.568949,1.317789,0.208308,-0.916677,-0.249703,-0.414735,-1.163082,-0.826724,-0.037048,-0.604404,-1.171858,0.408893,-0.369500,0.367665,-0.690134,-0.587932,1.021504,0.421772,0.419487,0.053271,0.003

In [ ]:
test_bert_cls

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,Labels Set
0,-0.885904,0.466816,-0.427689,-0.551959,0.533157,0.407121,0.677028,-0.669056,-0.898857,0.023043,0.299925,1.067225,0.065031,0.167264,0.285255,0.508361,0.874912,-0.435702,0.949470,-0.573988,-0.172619,0.491347,0.222306,-0.019993,0.262989,0.269541,0.064285,-0.265430,-0.416819,0.983758,0.180884,0.969636,-0.935556,0.376400,0.513621,0.103546,0.570385,-0.440762,0.492056,-0.153101,...,-0.413218,0.303507,0.077157,-0.530188,-0.656340,1.209545,0.413400,-0.434631,-0.084314,-0.565469,-0.501812,-0.780479,-0.451145,-0.623478,-0.888924,0.034785,-0.608549,0.441932,-0.674239,-0.470665,0.804236,0.450454,0.520809,0.031039,0.270253,-0.963066,0.139201,-0.520988,0.554184,-0.190132,-0.248794,-0.900229,-0.145893,0.420340,-0.839353,0.007705,0.284119,0.497907,-0.237154,"[1, 1, 0, 1, 0]"
1,-0.328235,0.636561,-0.446008,-0.356067,0.348313,0.636951,1.297130,-0.190477,-0.772605,0.370287,0.241562,0.935303,-0.253968,0.629992,0.444548,0.357012,0.968398,-0.298517,0.849445,-0.706392,-0.218161,0.457557,0.076207,0.283063,0.458753,0.648191,0.280102,-0.210396,-0.564550,0.263033,0.259871,0.412313,-0.535223,0.128898,0.832908,-0.224453,0.131025,-0.418456,0.007175,-0.108965,...,-0.739309,-0.368781,0.166737,-0.312203,-0.623050,1.459129,0.191375,-0.632179,-0.108748,-0.198011,-0.739645,-0.914802,-0.031963,-0.151015,-1.089221,0.138736,-0.112483,0.510915,-0.569364,-0.741172,0.504024,0.033575,0.531643,0.348544,0.040830,-1.091184,-0.205248,-0.558241,0.611692,-0.281311,-0.128061,-1.126138,-0.007551,-0.003804,-0.918339,-0.068674,-0.111705,0.345142,-0.085919,"[0, 0, 0, 0, 1]"
2,-0.115869,0.732450,-0.470565,-0.448217,0.459434,0.320139,1.555896,-0.557283,-0.636583,0.385315,0.298730,1.098178,-0.335207,0.481105,0.444446,0.386692,1.133096,-0.193877,0.884883,-0.605719,-0.275090,0.575825,0.230370,0.367972,0.580230,0.695793,0.354919,-0.541572,-0.773865,0.570455,0.501646,0.378046,-0.905853,0.301691,0.446072,-0.015504,0.252207,-0.597836,-0.030192,-0.386153,...,-0.579264,-0.618084,0.135210,-0.466657,-0.567463,1.227051,0.416720,-1.082812,-0.037445,-0.261764,-0.947571,-0.748804,-0.067237,-0.297960,-1.269735,0.074833,-0.405157,0.372466,-0.767093,-0.245691,0.833395,0.401494,0.785679,0.328373,-0.006599,-1.170845,-0.015948,-0.598315,0.590769,-0.007908,0.021510,-1.096594,-0.042052,0.126680,-0.978304,-0.003351,0.061333,0.570322,-0.136338,"[0, 0, 1, 0, 0]"
3,-0.072765,0.269666,-0.364946,-0.652143,0.526090,0.174933,1.427172,-0.413929,-0.294776,0.394405,0.372714,1.307387,-0.317260,0.463952,0.318658,0.346565,0.938303,-0.238120,0.463205,-0.555183,-0.399005,0.376617,0.172027,0.356471,0.371958,0.346057,0.367789,-0.695618,-0.706265,0.483107,0.432156,0.085090,-0.748587,0.171959,0.208016,0.060247,0.236989,-0.539263,-0.034663,-0.408213,...,-0.542544,-0.841446,0.286198,-0.633447,-0.350570,1.250170,0.446010,-1.096894,0.052925,-0.202299,-0.685216,-0.617260,-0.354695,-0.400166,-0.853179,-0.070509,-0.388231,0.399528,-0.677697,-0.070673,0.774395,0.109454,0.647744,0.187045,-0.093630,-1.173455,0.011168,-0.565113,0.700168,-0.003849,-0.143173,-0.915113,0.025286,-0.097589,-1.303863,0.236476,0.242160,0.400300,-0.316995,"[0, 1, 0, 0, 0]"
4,-0.520371,0.471309,-0.400060,-0.527720,0.593068,0.361313,1.112683,-0.275738,-0.412458,0.320607,0.301265,1.284021,-0.292908,0.260263,0.542277,0.387371,1.156700,-0.281170,0.423712,-0.793372,-0.356314,0.327025,-0.190835,0.238644,0.407796,0.535257,0.040089,-0.582761,-0.735752,0.601424,0.271761,0.548042,-0.932749,0.493943,0.639699,0.025410,0.224559,-0.495324,0.119329,0.116701,...,-0.613774,-0.295043,0.262508,-0.342428,-0.509207,1.479888,0.292177,-0.811918,-0.011435,-0.036894,-0.657376,-1.031175,-0.297405,-0.325522,-1.003590,0.152291,-0.288245,0.339992,-0.650380,-0.390734,0.783713,0.265524,0.780837,0.306837,0.003428,

In [ ]:
def create_multi_label_data(df):

    total_data = []
    for i in range(len(df)):
        sample = []
        labels = df['Labels Set'][i]

        if(labels[0]==1):
            sample.append(1)
        else:
            sample.append(0)


        if(labels[1]==1):
            sample.append(1)
        else:
            sample.append(0)


        if(labels[2]==1):
            sample.append(1)
        else:
            sample.append(0)
        

        if(labels[3]==1):
            sample.append(1)
        else:
            sample.append(0)
        

        if(labels[4]==1):
            sample.append(1)
        else:
            sample.append(0)

        
        total_data.append(sample)
        
    total_data = pd.DataFrame(total_data, columns = ['Defmatation','Fake','Hate','Offensive','Non-Hostile'])
    return total_data

In [ ]:


multi_train = create_multi_label_data(train_bert_cls)
train_bert_cls = train_bert_cls.drop(train_bert_cls.columns[-1],axis=1)
multi_train = pd.concat( [train_bert_cls,multi_train],axis=1)
multi_train



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,Defmatation,Fake,Hate,Offensive,Non-Hostile
0,-0.144744,0.753920,-0.370921,-0.611838,0.458139,0.191976,1.445014,-0.574958,-0.530926,0.478863,0.360358,1.185349,-0.117227,0.353543,0.289014,0.240586,0.918327,-0.305945,0.875681,-0.444604,-0.467921,0.422519,0.080013,0.269675,0.482003,0.725526,0.351850,-0.525961,-0.860559,0.508275,0.489205,0.122160,-0.546525,0.099349,0.583576,-0.010702,0.248857,-0.396506,-0.056162,-0.435263,...,-0.527120,1.364753,0.334724,-1.164227,-0.155990,-0.386536,-0.769295,-0.651128,-0.073439,-0.259624,-1.097570,-0.121238,-0.471517,0.338178,-0.808320,-0.136119,1.095405,0.175688,0.775150,0.348087,0.170919,-1.307852,-0.038388,-0.590353,0.536165,-0.006736,-0.119820,-0.907450,-0.134403,0.280859,-1.141616,0.078590,0.189792,0.589307,-0.206571,0,0,1,1,0
1,-0.519353,0.546359,-0.319189,-0.343406,0.505719,0.373533,1.298511,-0.344353,-0.530590,0.271636,0.292417,1.001168,-0.124076,0.185682,0.657651,0.287772,0.958941,-0.204964,0.836533,-0.753627,-0.514037,0.309644,0.023796,0.073000,0.524523,0.696313,0.141873,-0.547655,-0.631862,0.541703,0.285761,0.618741,-1.041801,0.418552,0.797897,0.144787,0.104293,-0.449161,-0.049456,-0.075865,...,-0.577727,1.469891,0.279616,-0.914689,-0.035396,-0.235962,-0.706077,-0.889545,0.015650,-0.178160,-0.755991,-0.091082,-0.379350,0.514821,-0.688070,-0.505213,0.880837,0.252368,0.674633,0.443868,-0.087992,-1.137267,-0.249685,-0.872141,0.609409,-0.215903,-0.286818,-0.965113,-0.154222,-0.008278,-0.969820,0.197929,0.145973,0.389509,-0.169360,0,0,0,0,1
2,-0.035937,0.593653,-0.028129,-0.481018,0.558263,0.194883,1.548601,-0.022254,-0.594360,0.296011,0.015633,1.396214,-0.270457,-0.021687,0.463896,0.366739,0.755320,-0.512736,0.867576,-0.751097,-0.319374,0.014895,0.010531,0.331701,0.465910,0.807952,0.127065,-0.334813,-0.748627,0.587821,0.072867,0.182084,-0.688813,0.271975,0.671757,0.292573,-0.198882,-0.176110,-0.048825,-0.067558,...,-0.755805,1.287324,0.211610,-0.989575,-0.142912,-0.291308,-0.932961,-0.552130,-0.138239,-0.629656,-1.007847,0.220976,-0.093494,0.453248,-1.047838,-0.551566,1.155589,0.186428,0.486323,0.205587,-0.226365,-1.205978,0.144138,-0.787474,0.391423,-0.125152,0.108841,-0.923864,-0.107842,-0.134367,-0.918103,0.002269,0.073970,0.471125,-0.300663,0,0,0,0,1
3,-0.330956,0.835987,-0.244033,-0.472148,0.451524,0.050984,1.388158,-0.680034,-0.341974,0.238754,0.445741,1.286209,-0.095728,0.354333,0.181936,0.264372,0.966280,-0.244654,0.609970,-0.467251,-0.329228,0.438032,0.184911,0.297807,0.631634,0.734913,0.083992,-0.646495,-0.963654,0.812570,0.447330,0.327190,-0.721536,0.113138,0.413360,0.232470,0.170344,-0.340805,0.104577,-0.505388,...,-0.529693,1.097748,0.288841,-1.240061,-0.046536,-0.321174,-0.795053,-0.756293,-0.318385,-0.488590,-1.142724,-0.139520,-0.622172,0.271261,-0.733643,-0.126173,1.047742,0.049972,0.877653,0.196111,0.136220,-1.186355,-0.022802,-0.559142,0.678789,0.160011,-0.122833,-1.104039,-0.117944,0.202165,-1.308372,0.128998,0.035356,0.759933,-0.158489,1,0,0,1,0
4,-0.572113,0.866568,-0.252675,-0.305324,0.361157,0.288053,1.026320,-0.391716,-0.831329,0.483821,0.015659,1.239518,-0.120939,0.212706,0.157652,0.540641,0.962199,-0.336899,0.931267,-0.577887,0.009949,0.440167,0.191770,0.207748,0.503330,0.932699,0.099071,-0.323963,-0.578352,0.675329,0.353921,0.533808,-0.687090,0.458181,0.901226,-0.141238,-0.074874,-0.561047,0.063945,-0.075129,...,-0.568949,1.317789,0.208308,-0.916677,-0.249703,-0.414735,-1.163082,-0.826724,-0.037048,-0.604404,-1.171858,0.408893,-0.369500,0.367665,-0.690134,-0.587932,1.021504,0.421772,0.419487,0.053271,0.003636,-1.351106,0.019663,-0.715910,0.153989,-0.202083,-0.325171,-1.044109,0.070099,0.399113,-1.023433,0.129222,0.222488,0.624688,0.062915,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [ ]:
multi_test = create_multi_label_data(test_bert_cls)
test_bert_cls = test_bert_cls.drop(test_bert_cls.columns[-1],axis=1)
multi_test = pd.concat( [test_bert_cls,multi_test],axis=1)
multi_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,Defmatation,Fake,Hate,Offensive,Non-Hostile
0,-0.885904,0.466816,-0.427689,-0.551959,0.533157,0.407121,0.677028,-0.669056,-0.898857,0.023043,0.299925,1.067225,0.065031,0.167264,0.285255,0.508361,0.874912,-0.435702,0.949470,-0.573988,-0.172619,0.491347,0.222306,-0.019993,0.262989,0.269541,0.064285,-0.265430,-0.416819,0.983758,0.180884,0.969636,-0.935556,0.376400,0.513621,0.103546,0.570385,-0.440762,0.492056,-0.153101,...,-0.656340,1.209545,0.413400,-0.434631,-0.084314,-0.565469,-0.501812,-0.780479,-0.451145,-0.623478,-0.888924,0.034785,-0.608549,0.441932,-0.674239,-0.470665,0.804236,0.450454,0.520809,0.031039,0.270253,-0.963066,0.139201,-0.520988,0.554184,-0.190132,-0.248794,-0.900229,-0.145893,0.420340,-0.839353,0.007705,0.284119,0.497907,-0.237154,1,1,0,1,0
1,-0.328235,0.636561,-0.446008,-0.356067,0.348313,0.636951,1.297130,-0.190477,-0.772605,0.370287,0.241562,0.935303,-0.253968,0.629992,0.444548,0.357012,0.968398,-0.298517,0.849445,-0.706392,-0.218161,0.457557,0.076207,0.283063,0.458753,0.648191,0.280102,-0.210396,-0.564550,0.263033,0.259871,0.412313,-0.535223,0.128898,0.832908,-0.224453,0.131025,-0.418456,0.007175,-0.108965,...,-0.623050,1.459129,0.191375,-0.632179,-0.108748,-0.198011,-0.739645,-0.914802,-0.031963,-0.151015,-1.089221,0.138736,-0.112483,0.510915,-0.569364,-0.741172,0.504024,0.033575,0.531643,0.348544,0.040830,-1.091184,-0.205248,-0.558241,0.611692,-0.281311,-0.128061,-1.126138,-0.007551,-0.003804,-0.918339,-0.068674,-0.111705,0.345142,-0.085919,0,0,0,0,1
2,-0.115869,0.732450,-0.470565,-0.448217,0.459434,0.320139,1.555896,-0.557283,-0.636583,0.385315,0.298730,1.098178,-0.335207,0.481105,0.444446,0.386692,1.133096,-0.193877,0.884883,-0.605719,-0.275090,0.575825,0.230370,0.367972,0.580230,0.695793,0.354919,-0.541572,-0.773865,0.570455,0.501646,0.378046,-0.905853,0.301691,0.446072,-0.015504,0.252207,-0.597836,-0.030192,-0.386153,...,-0.567463,1.227051,0.416720,-1.082812,-0.037445,-0.261764,-0.947571,-0.748804,-0.067237,-0.297960,-1.269735,0.074833,-0.405157,0.372466,-0.767093,-0.245691,0.833395,0.401494,0.785679,0.328373,-0.006599,-1.170845,-0.015948,-0.598315,0.590769,-0.007908,0.021510,-1.096594,-0.042052,0.126680,-0.978304,-0.003351,0.061333,0.570322,-0.136338,0,0,1,0,0
3,-0.072765,0.269666,-0.364946,-0.652143,0.526090,0.174933,1.427172,-0.413929,-0.294776,0.394405,0.372714,1.307387,-0.317260,0.463952,0.318658,0.346565,0.938303,-0.238120,0.463205,-0.555183,-0.399005,0.376617,0.172027,0.356471,0.371958,0.346057,0.367789,-0.695618,-0.706265,0.483107,0.432156,0.085090,-0.748587,0.171959,0.208016,0.060247,0.236989,-0.539263,-0.034663,-0.408213,...,-0.350570,1.250170,0.446010,-1.096894,0.052925,-0.202299,-0.685216,-0.617260,-0.354695,-0.400166,-0.853179,-0.070509,-0.388231,0.399528,-0.677697,-0.070673,0.774395,0.109454,0.647744,0.187045,-0.093630,-1.173455,0.011168,-0.565113,0.700168,-0.003849,-0.143173,-0.915113,0.025286,-0.097589,-1.303863,0.236476,0.242160,0.400300,-0.316995,0,1,0,0,0
4,-0.520371,0.471309,-0.400060,-0.527720,0.593068,0.361313,1.112683,-0.275738,-0.412458,0.320607,0.301265,1.284021,-0.292908,0.260263,0.542277,0.387371,1.156700,-0.281170,0.423712,-0.793372,-0.356314,0.327025,-0.190835,0.238644,0.407796,0.535257,0.040089,-0.582761,-0.735752,0.601424,0.271761,0.548042,-0.932749,0.493943,0.639699,0.025410,0.224559,-0.495324,0.119329,0.116701,...,-0.509207,1.479888,0.292177,-0.811918,-0.011435,-0.036894,-0.657376,-1.031175,-0.297405,-0.325522,-1.003590,0.152291,-0.288245,0.339992,-0.650380,-0.390734,0.783713,0.265524,0.780837,0.306837,0.003428,-0.969084,0.006941,-0.737195,0.499338,-0.247245,-0.147876,-0.828197,0.032513,-0.039999,-0.992171,0.089339,0.109733,0.370883,-0.104683,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
# #Dumping Embedding Features Multi Label DataFrame

# path_temp = r"/gdrive/My Drive/3rd Sem/MTech CSE 3/Thesis/IIITD Fake News Project/Data Set/Embeddings Datasets/BERT_CLS/multi_label_train_df.pkl"
# f = open(path_temp,'wb')
# pickle.dump(multi_train,f)
# f.close()


# path_temp = r"/gdrive/My Drive/3rd Sem/MTech CSE 3/Thesis/IIITD Fake News Project/Data Set/Embeddings Datasets/BERT_CLS/multi_label_test_df.pkl"
# f = open(path_temp,'wb')
# pickle.dump(multi_test,f)
# f.close()

In [ ]:
#Loading Embedding Features DataFrame

path_temp = r"/gdrive/My Drive/IIITD Fake News Project/Data Set/Embeddings Datasets/BERT_CLS/multi_label_train_df.pkl"
f = open(path_temp,'rb')
multi_train = pickle.load(f)
f.close()

path_temp = r"/gdrive/My Drive/IIITD Fake News Project/Data Set/Embeddings Datasets/BERT_CLS/multi_label_test_df.pkl"
f = open(path_temp,'rb')
multi_test = pickle.load(f)
f.close()


In [ ]:
multi_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,Defmatation,Fake,Hate,Offensive,Non-Hostile
0,-0.144744,0.753920,-0.370921,-0.611838,0.458139,0.191976,1.445014,-0.574958,-0.530926,0.478863,0.360358,1.185349,-0.117227,0.353543,0.289014,0.240586,0.918327,-0.305945,0.875681,-0.444604,-0.467921,0.422519,0.080013,0.269675,0.482003,0.725526,0.351850,-0.525961,-0.860559,0.508275,0.489205,0.122160,-0.546525,0.099349,0.583576,-0.010702,0.248857,-0.396506,-0.056162,-0.435263,...,-0.527120,1.364753,0.334724,-1.164227,-0.155990,-0.386536,-0.769295,-0.651128,-0.073439,-0.259624,-1.097570,-0.121238,-0.471517,0.338178,-0.808320,-0.136119,1.095405,0.175688,0.775150,0.348087,0.170919,-1.307852,-0.038388,-0.590353,0.536165,-0.006736,-0.119820,-0.907450,-0.134403,0.280859,-1.141616,0.078590,0.189792,0.589307,-0.206571,0,0,1,1,0
1,-0.519353,0.546359,-0.319189,-0.343406,0.505719,0.373533,1.298511,-0.344353,-0.530590,0.271636,0.292417,1.001168,-0.124076,0.185682,0.657651,0.287772,0.958941,-0.204964,0.836533,-0.753627,-0.514037,0.309644,0.023796,0.073000,0.524523,0.696313,0.141873,-0.547655,-0.631862,0.541703,0.285761,0.618741,-1.041801,0.418552,0.797897,0.144787,0.104293,-0.449161,-0.049456,-0.075865,...,-0.577727,1.469891,0.279616,-0.914689,-0.035396,-0.235962,-0.706077,-0.889545,0.015650,-0.178160,-0.755991,-0.091082,-0.379350,0.514821,-0.688070,-0.505213,0.880837,0.252368,0.674633,0.443868,-0.087992,-1.137267,-0.249685,-0.872141,0.609409,-0.215903,-0.286818,-0.965113,-0.154222,-0.008278,-0.969820,0.197929,0.145973,0.389509,-0.169360,0,0,0,0,1
2,-0.035937,0.593653,-0.028129,-0.481018,0.558263,0.194883,1.548601,-0.022254,-0.594360,0.296011,0.015633,1.396214,-0.270457,-0.021687,0.463896,0.366739,0.755320,-0.512736,0.867576,-0.751097,-0.319374,0.014895,0.010531,0.331701,0.465910,0.807952,0.127065,-0.334813,-0.748627,0.587821,0.072867,0.182084,-0.688813,0.271975,0.671757,0.292573,-0.198882,-0.176110,-0.048825,-0.067558,...,-0.755805,1.287324,0.211610,-0.989575,-0.142912,-0.291308,-0.932961,-0.552130,-0.138239,-0.629656,-1.007847,0.220976,-0.093494,0.453248,-1.047838,-0.551566,1.155589,0.186428,0.486323,0.205587,-0.226365,-1.205978,0.144138,-0.787474,0.391423,-0.125152,0.108841,-0.923864,-0.107842,-0.134367,-0.918103,0.002269,0.073970,0.471125,-0.300663,0,0,0,0,1
3,-0.330956,0.835987,-0.244033,-0.472148,0.451524,0.050984,1.388158,-0.680034,-0.341974,0.238754,0.445741,1.286209,-0.095728,0.354333,0.181936,0.264372,0.966280,-0.244654,0.609970,-0.467251,-0.329228,0.438032,0.184911,0.297807,0.631634,0.734913,0.083992,-0.646495,-0.963654,0.812570,0.447330,0.327190,-0.721536,0.113138,0.413360,0.232470,0.170344,-0.340805,0.104577,-0.505388,...,-0.529693,1.097748,0.288841,-1.240061,-0.046536,-0.321174,-0.795053,-0.756293,-0.318385,-0.488590,-1.142724,-0.139520,-0.622172,0.271261,-0.733643,-0.126173,1.047742,0.049972,0.877653,0.196111,0.136220,-1.186355,-0.022802,-0.559142,0.678789,0.160011,-0.122833,-1.104039,-0.117944,0.202165,-1.308372,0.128998,0.035356,0.759933,-0.158489,1,0,0,1,0
4,-0.572113,0.866568,-0.252675,-0.305324,0.361157,0.288053,1.026320,-0.391716,-0.831329,0.483821,0.015659,1.239518,-0.120939,0.212706,0.157652,0.540641,0.962199,-0.336899,0.931267,-0.577887,0.009949,0.440167,0.191770,0.207748,0.503330,0.932699,0.099071,-0.323963,-0.578352,0.675329,0.353921,0.533808,-0.687090,0.458181,0.901226,-0.141238,-0.074874,-0.561047,0.063945,-0.075129,...,-0.568949,1.317789,0.208308,-0.916677,-0.249703,-0.414735,-1.163082,-0.826724,-0.037048,-0.604404,-1.171858,0.408893,-0.369500,0.367665,-0.690134,-0.587932,1.021504,0.421772,0.419487,0.053271,0.003636,-1.351106,0.019663,-0.715910,0.153989,-0.202083,-0.325171,-1.044109,0.070099,0.399113,-1.023433,0.129222,0.222488,0.624688,0.062915,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [ ]:
multi_test

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,Defmatation,Fake,Hate,Offensive,Non-Hostile
0,-0.885904,0.466816,-0.427689,-0.551959,0.533157,0.407121,0.677028,-0.669056,-0.898857,0.023043,0.299925,1.067225,0.065031,0.167264,0.285255,0.508361,0.874912,-0.435702,0.949470,-0.573988,-0.172619,0.491347,0.222306,-0.019993,0.262989,0.269541,0.064285,-0.265430,-0.416819,0.983758,0.180884,0.969636,-0.935556,0.376400,0.513621,0.103546,0.570385,-0.440762,0.492056,-0.153101,...,-0.656340,1.209545,0.413400,-0.434631,-0.084314,-0.565469,-0.501812,-0.780479,-0.451145,-0.623478,-0.888924,0.034785,-0.608549,0.441932,-0.674239,-0.470665,0.804236,0.450454,0.520809,0.031039,0.270253,-0.963066,0.139201,-0.520988,0.554184,-0.190132,-0.248794,-0.900229,-0.145893,0.420340,-0.839353,0.007705,0.284119,0.497907,-0.237154,1,1,0,1,0
1,-0.328235,0.636561,-0.446008,-0.356067,0.348313,0.636951,1.297130,-0.190477,-0.772605,0.370287,0.241562,0.935303,-0.253968,0.629992,0.444548,0.357012,0.968398,-0.298517,0.849445,-0.706392,-0.218161,0.457557,0.076207,0.283063,0.458753,0.648191,0.280102,-0.210396,-0.564550,0.263033,0.259871,0.412313,-0.535223,0.128898,0.832908,-0.224453,0.131025,-0.418456,0.007175,-0.108965,...,-0.623050,1.459129,0.191375,-0.632179,-0.108748,-0.198011,-0.739645,-0.914802,-0.031963,-0.151015,-1.089221,0.138736,-0.112483,0.510915,-0.569364,-0.741172,0.504024,0.033575,0.531643,0.348544,0.040830,-1.091184,-0.205248,-0.558241,0.611692,-0.281311,-0.128061,-1.126138,-0.007551,-0.003804,-0.918339,-0.068674,-0.111705,0.345142,-0.085919,0,0,0,0,1
2,-0.115869,0.732450,-0.470565,-0.448217,0.459434,0.320139,1.555896,-0.557283,-0.636583,0.385315,0.298730,1.098178,-0.335207,0.481105,0.444446,0.386692,1.133096,-0.193877,0.884883,-0.605719,-0.275090,0.575825,0.230370,0.367972,0.580230,0.695793,0.354919,-0.541572,-0.773865,0.570455,0.501646,0.378046,-0.905853,0.301691,0.446072,-0.015504,0.252207,-0.597836,-0.030192,-0.386153,...,-0.567463,1.227051,0.416720,-1.082812,-0.037445,-0.261764,-0.947571,-0.748804,-0.067237,-0.297960,-1.269735,0.074833,-0.405157,0.372466,-0.767093,-0.245691,0.833395,0.401494,0.785679,0.328373,-0.006599,-1.170845,-0.015948,-0.598315,0.590769,-0.007908,0.021510,-1.096594,-0.042052,0.126680,-0.978304,-0.003351,0.061333,0.570322,-0.136338,0,0,1,0,0
3,-0.072765,0.269666,-0.364946,-0.652143,0.526090,0.174933,1.427172,-0.413929,-0.294776,0.394405,0.372714,1.307387,-0.317260,0.463952,0.318658,0.346565,0.938303,-0.238120,0.463205,-0.555183,-0.399005,0.376617,0.172027,0.356471,0.371958,0.346057,0.367789,-0.695618,-0.706265,0.483107,0.432156,0.085090,-0.748587,0.171959,0.208016,0.060247,0.236989,-0.539263,-0.034663,-0.408213,...,-0.350570,1.250170,0.446010,-1.096894,0.052925,-0.202299,-0.685216,-0.617260,-0.354695,-0.400166,-0.853179,-0.070509,-0.388231,0.399528,-0.677697,-0.070673,0.774395,0.109454,0.647744,0.187045,-0.093630,-1.173455,0.011168,-0.565113,0.700168,-0.003849,-0.143173,-0.915113,0.025286,-0.097589,-1.303863,0.236476,0.242160,0.400300,-0.316995,0,1,0,0,0
4,-0.520371,0.471309,-0.400060,-0.527720,0.593068,0.361313,1.112683,-0.275738,-0.412458,0.320607,0.301265,1.284021,-0.292908,0.260263,0.542277,0.387371,1.156700,-0.281170,0.423712,-0.793372,-0.356314,0.327025,-0.190835,0.238644,0.407796,0.535257,0.040089,-0.582761,-0.735752,0.601424,0.271761,0.548042,-0.932749,0.493943,0.639699,0.025410,0.224559,-0.495324,0.119329,0.116701,...,-0.509207,1.479888,0.292177,-0.811918,-0.011435,-0.036894,-0.657376,-1.031175,-0.297405,-0.325522,-1.003590,0.152291,-0.288245,0.339992,-0.650380,-0.390734,0.783713,0.265524,0.780837,0.306837,0.003428,-0.969084,0.006941,-0.737195,0.499338,-0.247245,-0.147876,-0.828197,0.032513,-0.039999,-0.992171,0.089339,0.109733,0.370883,-0.104683,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

## Creating Multi-Label Models

##SVM Classifier

In [ ]:
multi_train = np.asarray(multi_train)
multi_test  = np.asarray(multi_test)

x_train = multi_train[:,:-5]
y_train = multi_train[:,-5:]

x_test  = multi_test[:,:-5]
y_test  = multi_test[:,-5:]


print("x_train: ",x_train.shape)
print("y_train",y_train.shape)
print("x_test",x_test.shape)
print("y_test",y_test.shape)

x_train:  (6539, 768)
y_train (6539, 5)
x_test (1653, 768)
y_test (1653, 5)


In [ ]:
# param_grid = {'C': [0.01, 0.1, 1, 10],
#               'gamma': [1,0.1],
#               'kernel': ['rbf','linear']}


# grid = GridSearchCV(SVC(),param_grid,verbose=10)
# grid.fit(x_train,y_train[:,0])
# print(grid.best_params_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=0.01, gamma=1, kernel=rbf .....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ......... C=0.01, gamma=1, kernel=rbf, score=0.902, total=  35.5s
[CV] C=0.01, gamma=1, kernel=rbf .....................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   35.5s remaining:    0.0s


[CV] ......... C=0.01, gamma=1, kernel=rbf, score=0.902, total=  35.5s
[CV] C=0.01, gamma=1, kernel=rbf .....................................


...

[CV] ...... C=0.01, gamma=1, kernel=linear, score=0.901, total=  12.6s
[CV] C=0.01, gamma=1, kernel=linear ..................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  3.8min remaining:    0.0s


[CV] ...... C=0.01, gamma=1, kernel=linear, score=0.902, total=  12.5s
[CV] C=0.01, gamma=0.1, kernel=rbf ...................................
[CV] ....... C=0.01, gamma=0.1, kernel=rbf, score=0.902, total=  22.2s
[CV] C=0.01, gamma=0.1, kernel=rbf ...................................
[CV] ....... C=0.01, gamma=0.1, kernel=rbf, score=0.902, total=  22.2s
[CV] C=0.01, gamma=0.1, kernel=rbf ...................................
[CV] ....... C=0.01, gamma=0.1, kernel=rbf, score=0.902, total=  22.1s
[CV] C=0.01, gamma=0.1, kernel=rbf ...................................
[CV] ....... C=0.01, gamma=0.1, kernel=rbf, score=0.901, total=  21.5s
[CV] C=0.01, gamma=0.1, kernel=rbf ...................................
[CV] ....... C=0.01, gamma=0.1, kernel=rbf, score=0.902, total=  22.2s
[CV] C=0.01, gamma=0.1, kernel=linear ................................
[CV] .... C=0.01, gamma=0.1, kernel=linear, score=0.902, total=  12.7s
[CV] C=0.01, gamma=0.1, kernel=linear ................................
[CV] .

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed: 32.6min finished


{'C': 1, 'gamma': 1, 'kernel': 'rbf'}


In [ ]:
sum(y_train[:,0])

641.0

In [ ]:
defamation_clf = SVC(kernel='rbf',C=1,gamma=1,verbose=True) ### Earlier C=1, gamma=0.01 gave 76% accuracy
defamation_clf.fit(x_train, y_train[:,0])

[LibSVM]

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=True)

In [ ]:
#Dumping the SVM Model
joblib.dump(defamation_clf,r"/gdrive/My Drive/IIITD Fake News Project/Notebooks/Models/SVM BERT Models/defamation.pkl")
defamation_clf = joblib.load(r"/gdrive/My Drive/IIITD Fake News Project/Notebooks/Models/SVM BERT Models/defamation.pkl")

In [ ]:
print("\n\nAccuracy report: ")
predict = defamation_clf.predict(x_test)
print(classification_report(predict,y_test[:,0]))
print(np.unique(predict,return_counts = True))



Accuracy report: 
              precision    recall  f1-score   support

         0.0       1.00      0.90      0.95      1645
         1.0       0.03      0.62      0.06         8

    accuracy                           0.90      1653
   macro avg       0.51      0.76      0.50      1653
weighted avg       0.99      0.90      0.94      1653

(array([0., 1.]), array([1645,    8]))


## Fake Model

In [ ]:
fake_clf = SVC(kernel='rbf',C=1,gamma=1,verbose=True) ### Earlier C=1, gamma=0.01 gave 76% accuracy
fake_clf.fit(x_train, y_train[:,1])

[LibSVM]

SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=True)

In [ ]:
#Dumping the SVM Model
joblib.dump(defamation_clf,r"/gdrive/My Drive/IIITD Fake News Project/Notebooks/Models/SVM BERT Models/fake.pkl")
defamation_clf = joblib.load(r"/gdrive/My Drive/IIITD Fake News Project/Notebooks/Models/SVM BERT Models/fake.pkl")

In [ ]:
print("\n\nAccuracy report: ")
predict = fake_clf.predict(x_test)
print(classification_report(predict,y_test[:,1]))
print(np.unique(predict,return_counts = True))



Accuracy report: 
              precision    recall  f1-score   support

         0.0       1.00      0.80      0.89      1644
         1.0       0.02      0.89      0.05         9

    accuracy                           0.80      1653
   macro avg       0.51      0.85      0.47      1653
weighted avg       0.99      0.80      0.89      1653

(array([0., 1.]), array([1644,    9]))


In [ ]:
pip install scikit-multilearn

     |████████████████████████████████| 92kB 4.2MB/s 


In [ ]:
# Import BinaryRelevance from skmultilearn
from skmultilearn.problem_transform import BinaryRelevance

# Import SVC classifier from sklearn
from sklearn.svm import SVC

# Setup the classifier
classifier = BinaryRelevance(classifier=SVC(), require_dense=[False,True])

# Train
classifier.fit(x_train, y_train)
# Predict
y_pred = classifier.predict(x_test)

In [ ]:
print(y_pred[39])

  (0, 4)	1.0


In [ ]:
from skmultilearn.adapt import MLkNN 

mlknn_classifier = MLkNN() 
mlknn_classifier.fit(x_train,y_train) 


MLkNN(ignore_first_neighbours=0, k=10, s=1.0)

In [ ]:
y_pred = mlknn_classifier.predict(x_test) 
print(y_pred.toarray())

[[0 0 0 0 1]
 [0 0 0 0 1]
 [0 1 0 0 0]
 ...
 [0 0 0 1 0]
 [0 0 0 0 1]
 [1 0 0 1 0]]


In [ ]:
y_pred.shape

(1653, 5)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score,accuracy_score
from sklearn.metrics import classification_report


r  = recall_score(y_true=y_test, y_pred=y_pred, average='weighted')

p  = precision_score(y_true=y_test, y_pred=y_pred, average='weighted')

f1 = f1_score(y_true=y_test, y_pred=y_pred, average='weighted')

# print(p)
# print(r)
# print(f1)
# print(accuracy_score(y_test,y_pred))
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.27      0.10      0.15       169
           1       0.41      0.19      0.26       334
           2       0.36      0.13      0.19       234
           3       0.39      0.29      0.33       219
           4       0.77      0.80      0.78       873

   micro avg       0.63      0.48      0.54      1829
   macro avg       0.44      0.30      0.34      1829
weighted avg       0.56      0.48      0.50      1829
 samples avg       0.51      0.51      0.51      1829



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
